In [1]:
#Libraries used in  part 1, 2 and 3
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
from math import pi

#Libraries used for bookeh
from bokeh.models import LabelSet, ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.models import Legend
from bokeh.io import output_notebook
from bokeh.palettes import Category20c
from bokeh.transform import cumsum

#Libraries used for Folium
import folium
import json

#Libraries for interaction
import ipywidgets

output_notebook()
%matplotlib inline


#data_1 downloaded from https://admin.opendata.dk/dataset/1922db99-025b-4974-a60b-a19e3605d660/resource/359328fa-61e8-4e11-afbf-16686b98542a/download/openenergydays16.08.2016.xlsx
df1 = pd.read_excel("C:/Users/alikuc/Desktop/PhD/Course/02806 - Social data analysis and visualisation/Assignments/Final assignment/openenergydays16.xlsx") 

Loading BokehJS ...

# Introduction

The purpose of this article is to analyze and visualize 10 areas in Copenhagen with the greatest heat saving potential and define its target group. This article is aimed at manufacturers with heat-saving products that manufacturers can use to target their marketing efforts in Copenhagen.

The 10 areas in Copenhagen are:
* Vesterbro-Kongens Enghave
* Bispebjerg
* Østerbro
* Indre by
* Vest Amager
* Øst Amager
* Nørrebro
* Valby
* Vanløse
* Brønshøj-Husum



## Private owners have the highest heat consumption

The total heat consumption in the selected 10 areas in Copenhagen is 4740.02 GWh. The distribution of heat consumption shows that private individuals or partnerships is the largest consumer, accounting for 72% of the total heat consumption. Private cooperative housing associations is taking the second place, accounting for 9% of the total heat consumption. Private owners is therefore representing 81% of the heat consumption in Copenhagen.  

In [2]:
#CLEANING DATA 1
#Checking if there are empty or nan values
np.where(pd.isnull(df1))

# Removing 0 from samlet_opvarmet_areal column.
df1 = df1.loc[df1["samlet_opvarmet_areal"] >0]

# Removing districts, which does not have information about population
df1 = df1.loc[(df1["postbynavn"] != "Hellerup") & (df1["postbynavn"] != "Kastrup") & (df1["postbynavn"] != "Frederiksberg")& (df1["postbynavn"] != "Nordhavn")& (df1["postbynavn"] != "Rødovre")& (df1["postbynavn"] != "Søborg")]



__The distribution of heat consumption based on ownership ratio as a percentage.__

In [3]:
# Calculating the sum of heat consumption for Copenhagen districts
heatConsumptionSumKWh = df1["Teoretisk SBI samlet_varmeforbrug (KWh)"].sum().round(0)
heatConsumptionSumGWh =(heatConsumptionSumKWh/1000000).round(2)

# Grouping the list by ejerforhold and resetting the index number
df1C = df1.groupby(["ejerforhold"]).sum().reset_index()

# Changing the column name for heat consumption to 1 string, so that it can work in tootips later below
df1C = df1C.rename(columns={"Teoretisk SBI samlet_varmeforbrug (KWh)": "Heat_Consumption"})

# Changing the frame to only contain ejerforhold and Heat_Consumption
df1C = df1C[["ejerforhold","Heat_Consumption"]]

# Calculating the percentage
df1C["Heat_Consumption"] = (df1C["Heat_Consumption"]/(df1C["Heat_Consumption"].sum())*100).round(0)

# Calculating the angle, which will be used for the pie
df1C['angle'] = df1C["Heat_Consumption"]/df1C["Heat_Consumption"].sum() * 2*pi

# Defining a color for each ejerforhold
df1C['color'] = Category20c[len(df1C)]

# Creating the figure
p = figure(plot_height=500,plot_width=900, toolbar_location=None,
           tools="hover", tooltips="@ejerforhold: @Heat_Consumption %", x_range=(-0.5, 1.0))

# Creating the pie chart
p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='ejerforhold', source=ColumnDataSource(df1C))

# Changing the type from float to int64
df1C["Heat_Consumption"]= df1C["Heat_Consumption"].astype(np.int64)

# Changing the type to string
df1C["Heat_Consumption"] = df1C["Heat_Consumption"].astype(str)

# Adding % to string 
df1C["Heat_Consumption"] = df1C["Heat_Consumption"] +str("%")

# Moving the string in the Pie Chart with 25 from left
df1C["Heat_Consumption"] = df1C["Heat_Consumption"].str.pad(25, side = "left")

# Defining labels
labels = LabelSet(x=0, y=1, text='Heat_Consumption',
        angle=cumsum('angle', include_zero=True), text_color="black",
          source=ColumnDataSource(df1C), render_mode='canvas')

# Adding labels and changing setting for the figure
p.add_layout(labels) 
p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None
p.outline_line_color=None
p.legend.border_line_color = None

# Displaying the figure
show(p)

## Brønshøj-Husum has the highest sales potential as location in Copenhagen area

With a heat consumption between 108-112 kWh / m2, Vesterbro-Kongens Enghave and Nørrebro have the lowest consumption per square meter compared to the other areas in Copenhagen. With a heat consumption between 108 to 112 kWh / m2, Brønshøj-Husum has the highest consumption per square meter compared to the other areas in Copenhagen, closely followed by Vanløse, Valby, Vest Amager and Øst Amager.

__OVERSKRIFT FOR INTERACTIVE MAPS__

In [8]:
# INTERACTIVE MAPS CODE

# DATAFRAME CODE
#Grouping by district
df1M = df1.groupby(["postbynavn"]).sum().reset_index()

# Calculating heat consumption per m2 for each city
df1M["heatPerArea"] = (df1M["Teoretisk SBI samlet_varmeforbrug (KWh)"]/df1M["samlet_opvarmet_areal"]).astype(int)

#Changing names so that it corresponds to the geojson file
df1M.iloc[0,0] ="Brønshøj-Husum" 
df1M.iloc[1,0] ="Indre By" 
df1M.iloc[2,0] ="Nørrebro" 
df1M.iloc[3,0] ="Bispebjerg" 
df1M.iloc[4,0] ="Amager Øst" 
df1M.iloc[5,0] ="Amager Vest" 
df1M.iloc[6,0] ="Vesterbro-Kongens Enghave" 
df1M.iloc[7,0] ="Østerbro" 
df1M.iloc[8,0] ="Valby" 
df1M.iloc[9,0] ="Vanløse" 

# Creating a new dataframe, with only 2 columns
df1M3 = df1M[["postbynavn","heatPerArea"]]

#GEOJSON CODE
#Importing the geojson file found from https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/copenhagen.geojson
geoJSON = pd.read_json("C:/Users/alikuc/Desktop/PhD/Course/02806 - Social data analysis and visualisation/Assignments/Final assignment/copenhagen.geojson")

# Creating a list based on geojson file
listA=[]
for i in geoJSON.features.keys():
    listA.append(geoJSON.features[i])

# Changing the order of the city data in geojson file so that it corresponds to the order of city data in the new dataframe
listB=[listA[5],listA[0],listA[6],listA[7],listA[4],listA[9],listA[3],listA[1],listA[2],listA[8]]
listA[9]["properties"]["name"]


# FOLIUM CODE
# Using folium method with longitude, latitude and starting the zoom level at 11
m = folium.Map(location=[55.676098, 12.568337], zoom_start=11)

# Changing the color palette scale
myscale = (df1M3['heatPerArea'].quantile((0,0.2,0.3,0.4,0.53,0.89,1))).tolist()

def map_cities(x):
    m.choropleth(
    geo_data=listB[x],
    name=df1M3.iloc[x,0],
    data=df1M3,
    columns=['postbynavn', 'heatPerArea'],
    key_on="properties.name",
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="KWh/m2",
    threshold_scale=myscale,
        
    )
    
    style_function = lambda x: {'fillColor': '#ffffff', 
                                'color':'#000000', 
                                'fillOpacity': 0.1, 
                                'weight': 0.1}  

    highlight_function = lambda x: {'fillColor': '#000000', 
                                    'color':'#000000', 
                                    'fillOpacity': 0.50, 
                                    'weight': 0.1}
    NIL = folium.features.GeoJson(
        listB[x],
        style_function=style_function, 
        control=False,
        highlight_function=highlight_function, 
        tooltip=folium.features.GeoJsonTooltip(
            fields=['name'],
            aliases=['City Name: '],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
        )
    )

    m.add_child(NIL)
    m.keep_in_front(NIL)

citiesArray=[0,1,2,3,4,5,6,7,8,9]
for i in citiesArray:
    map_cities(i)

# Plot it on the map
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)
folium.map.LayerControl('topright', collapsed=False).add_to(m)

# Displaying the map
m


##  People who lives without a partner in the area is consuming more heat

The most heat-consuming city Brønshøj-Husum has a civil state of 23% singles, 22% divorced, 22% married, 19% widowed, 9% registrered partnership, 4% dissolved partnerhip and 1% longest living partner. Thus, the majority, ie __69% of the population group lives in Brønshøj-Husum without a partner today__.

__Civil status for each city__

In [5]:
#CLEANING DATA 2
#data_1 downloaded from https://admin.opendata.dk/dataset/1922db99-025b-4974-a60b-a19e3605d660/resource/359328fa-61e8-4e11-afbf-16686b98542a/download/openenergydays16.08.2016.csv
df2 = pd.read_csv("C:/Users/alikuc/Desktop/PhD/Course/02806 - Social data analysis and visualisation/Assignments/Final assignment/befkbhalderkoencivst.csv") 

#Checking if there are empty or nan values
np.where(pd.isnull(df2))

# Removing districts, which does not have information about population
df2 = df2.loc[(df2["AAR"] == 2015) & (df2["BYDEL"] != 99)]

#Dropping AAR column
df2.drop(["AAR"], axis=1)

#Changing BYDEL number to names as mentioned in https://www.opendata.dk/city-of-copenhagen/befolkningen-efter-ar-bydel-alder-kon-og-civilstand
df2["BYDEL"] = df2['BYDEL'].replace({1: "Indre By"})
df2["BYDEL"] = df2['BYDEL'].replace({2: "Østerbro"})
df2["BYDEL"] = df2['BYDEL'].replace({3: "Nørrebro"})
df2["BYDEL"] = df2['BYDEL'].replace({4: "Vesterbro-Kongens Enghave"})
df2["BYDEL"] = df2['BYDEL'].replace({5: "Valby"})
df2["BYDEL"] = df2['BYDEL'].replace({6: "Vanløse"})
df2["BYDEL"] = df2['BYDEL'].replace({7: "Brønshøj-Husum"})
df2["BYDEL"] = df2['BYDEL'].replace({8: "Bispebjerg"})
df2["BYDEL"] = df2['BYDEL'].replace({9: "Amager Øst"})
df2["BYDEL"] = df2['BYDEL'].replace({10: "Amager Vest"})

#Changing KOEN to male/female as mentioned in https://www.opendata.dk/city-of-copenhagen/befolkningen-efter-ar-bydel-alder-kon-og-civilstand
df2["KOEN"] = df2['KOEN'].replace({1: "Male"})
df2["KOEN"] = df2['KOEN'].replace({2: "Female"})

#Changing CIVST to male/female as mentioned in https://www.opendata.dk/city-of-copenhagen/befolkningen-efter-ar-bydel-alder-kon-og-civilstand
df2["CIVST"] = df2['CIVST'].replace({"E": "Widow"})
df2["CIVST"] = df2['CIVST'].replace({"F": "Divorced"})
df2["CIVST"] = df2['CIVST'].replace({"G": "Married"})
df2["CIVST"] = df2['CIVST'].replace({"L": "Longest-living partner"})
df2["CIVST"] = df2['CIVST'].replace({"O": "Dissolved partnership"})
df2["CIVST"] = df2['CIVST'].replace({"P": "Registered partnership"})
df2["CIVST"] = df2['CIVST'].replace({"U": "Single"})

In [6]:
@ipywidgets.interact
def plot(City=["Indre By", "Østerbro","Nørrebro","Vesterbro-Kongens Enghave" ,"Valby" ,"Vanløse" ,"Brønshøj-Husum", "Bispebjerg","Amager Øst" ,"Amager Vest"]):
    
    # Removing row with age below 18
    df2C2 = df2.loc[(df2["ALDER"] >= 18)]
    
    #Grouping by city and civil status
    df2C2 =  df2C2.groupby(["BYDEL", "CIVST"]).size().reset_index()
    df2C2 = df2C2.rename(columns={0: "Amount"})

    # Grouping the list by civil status and resetting the index number
    df2C2=df2C2.loc[(df2C2["BYDEL"] == City)] 
    df2C2 = df2C2.groupby(["CIVST"]).sum().reset_index()

    # Calculating the percentage
    df2C2["CivilPERCENTAGE"] = (df2C2["Amount"]/(df2C2["Amount"].sum())*100).round(0)

    # Calculating the angle, which will be used for the pie
    df2C2['angle'] = df2C2["CivilPERCENTAGE"]/df2C2["CivilPERCENTAGE"].sum() * 2*pi

    # Defining a color for each civil status
    df2C2['color'] = Category20c[len(df2C2)]

    # Creating the figure
    p1 = figure(plot_height=500,plot_width=900, toolbar_location=None,
               tools="hover", tooltips="@CIVST: @CivilPERCENTAGE %", x_range=(-0.5, 1.0))

    # Creating the pie chart
    p1.wedge(x=0, y=1, radius=0.4,
            start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
            line_color="white", fill_color='color', legend_field='CIVST', source=ColumnDataSource(df2C2))

    # Changing the type from float to int64
    df2C2["CivilPERCENTAGE"]= df2C2["CivilPERCENTAGE"].astype(np.int64)

    # Changing the type to string
    df2C2["CivilPERCENTAGE"] = df2C2["CivilPERCENTAGE"].astype(str)

    # Adding % to string 
    df2C2["CivilPERCENTAGE"] = df2C2["CivilPERCENTAGE"] +str("%")

    # Moving the string in the Pie Chart with 25 from left
    df2C2["CivilPERCENTAGE"] = df2C2["CivilPERCENTAGE"].str.pad(25, side = "left")

    # Defining labels
    labels = LabelSet(x=0, y=1, text='CivilPERCENTAGE',
            angle=cumsum('angle', include_zero=True), text_color="black",
              source=ColumnDataSource(df2C2), render_mode='canvas')

    # Adding labels and changing setting for the figure
    p1.add_layout(labels) 
    p1.axis.axis_label=None
    p1.axis.visible=False
    p1.grid.grid_line_color = None
    p1.outline_line_color=None
    p1.legend.border_line_color = None

    # Displaying the figure
    show(p1)




interactive(children=(Dropdown(description='City', options=('Indre By', 'Østerbro', 'Nørrebro', 'Vesterbro-Kon…

## The largest age group is 21-30 years old

Generally, for all cities, the largest age group is 21-30 years. In Brønshøj-Husum, the number of people in the age group 21-30 is 7,031 people, closely followed by the age group 31-40 years with 6432 people and 41-50 years with 6038 people.


__Age groups for each city__

In [7]:
# Removing row with age below 18
df2C3 = df2.loc[(df2["ALDER"] >= 18)]

#Grouping by city and civil status
df2C3 = df2C3.groupby(["BYDEL", "ALDER"]).sum().reset_index()

#Categorizing age
df2C3["AgeGroup"]=""
for i, row in df2C3.iterrows():
    if row["ALDER"] > 17 and row["ALDER"] < 21:
        row["ALDER"] = "18-20"
        df2C3.at[i,'AgeGroup'] = row["ALDER"]
        
    elif row["ALDER"] > 20 and row["ALDER"] < 31:
        row["ALDER"] = "21-30"
        df2C3.at[i,'AgeGroup'] = row["ALDER"]
        
    elif row["ALDER"] > 30 and row["ALDER"] < 41:
        row["ALDER"] = "31-40"
        df2C3.at[i,'AgeGroup'] = row["ALDER"]

    elif row["ALDER"] > 40 and row["ALDER"] < 51:
        row["ALDER"] = "41-50"
        df2C3.at[i,'AgeGroup'] = row["ALDER"]
        
    elif row["ALDER"] > 50 and row["ALDER"] < 61:
        row["ALDER"] = "51-60"
        df2C3.at[i,'AgeGroup'] = row["ALDER"]

    elif row["ALDER"] > 60 and row["ALDER"] < 71:
        row["ALDER"] = "61-70"
        df2C3.at[i,'AgeGroup'] = row["ALDER"]
        
    elif row["ALDER"] > 70 and row["ALDER"] < 81:
        row["ALDER"] = "71-80"
        df2C3.at[i,'AgeGroup'] = row["ALDER"]
        
    elif row["ALDER"] > 80 and row["ALDER"] < 91:
        row["ALDER"] = "81-90"
        df2C3.at[i,'AgeGroup'] = row["ALDER"]
        
    elif row["ALDER"] > 90:
        row["ALDER"] = "90+"
        df2C3.at[i,'AgeGroup'] = row["ALDER"]
        
#Grouping by city and age category and getting the sum 
df2C3=df2C3.groupby(["BYDEL", "AgeGroup"]).sum().reset_index()

#Creating interactive plot
@ipywidgets.interact
def plot(City=["Indre By", "Østerbro","Nørrebro","Vesterbro-Kongens Enghave" ,"Valby" ,"Vanløse" ,"Brønshøj-Husum", "Bispebjerg","Amager Øst" ,"Amager Vest"]):
   
    df2C4=df2C3.loc[(df2C3["BYDEL"] == City)] 
    source=ColumnDataSource(data=df2C4)
    
    # Creating the figure
    p = figure(x_range=df2C4["AgeGroup"], plot_height=300, plot_width=800,
           toolbar_location=None, tools="hover",tooltips="@PERSONER People",)
    
    # Creating the vbar
    p.vbar(x="AgeGroup", top="PERSONER", width=0.6, muted_alpha=0.3, source=source)
    
    # Adding labels and changing setting for the figure
    p.y_range.start = 0
    p.xgrid.grid_line_color = None
    p.xaxis.axis_label = 'Group of age'
    p.yaxis.axis_label = 'Amount of people'
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.outline_line_color=None
   
    # Displaying the figure
    show(p)

interactive(children=(Dropdown(description='City', options=('Indre By', 'Østerbro', 'Nørrebro', 'Vesterbro-Kon…

##  CONCLUSION


It can be concluded that greatest heat saving potential in Copenhagen is placed in Brønshøj-Husum and Private owners own 72% of the buildings in Copenhagen. The majority of the civil status, i.e. 69% of the people in Husum-Brønshøj is living without a partner and the largest age group is 21-30 years, with 7.031 people. It can also be concluded that Vanløse, Valby, Øst Amager and Vest Amager are the second most heat consuming areas per square meter in Copenhagen and has the same target group as Brønshøj-Husum. If any sales strategy is developed for Brønshøj-Husum, it can also be applied for those cities.  